# Winery classification using the one-dimensional Gaussian

The **Wine** data set contains 178 labeled data points, each corresponding to a bottle of wine:
* The features (`x`): a 13-dimensional vector consisting of visual and chemical features for the bottle of wine
* The label (`y`): the winery from which the bottle came (1,2 or 3)

The data can be downloaded from the UCI repository (https://archive.ics.uci.edu/ml/datasets/wine).

## Load in the data set

We start by loading the packages we will need.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Useful module for dealing with the Gaussian density
from scipy.stats import norm, multivariate_normal

# installing packages for interactive graphs
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider

Next, we load the Wine data set and divide these into a training set of 130 points and a test set of 48 points.

In [2]:
data = np.loadtxt('wine.data.txt', delimiter=',')

# Names of features
featurenames = ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash','Magnesium', 'Total phenols', 
                'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 
                'OD280/OD315 of diluted wines', 'Proline']

Fix a particular "random" permutation of the data, and use these to effect the training / test split.

In [3]:
# Split 178 instances into training set (trainx, trainy) of size 130 and test set (testx, testy) of size 48
# Also split apart data and labels
np.random.seed(0)
perm = np.random.permutation(178)
trainx = data[perm[0:130],1:14]
trainy = data[perm[0:130],0]
testx = data[perm[130:178], 1:14]
testy = data[perm[130:178],0]

We get four arrays:
* `trainx`: 130x13, the training points
* `trainy`: 130x1, labels of the training points
* `testx`: 48x13, the test points
* `testy`: 48x1, labels of the test points

Let's see how many training points there are from each class.

In [4]:
sum(trainy==1), sum(trainy==2), sum(trainy==3)

(43, 54, 33)

## Look at the distribution of a single feature from one of the wineries

Let's pick just one feature: 'Alcohol'. This is the first feature. Here is a *histogram* of this feature's values under class 1, along with the *Gaussian fit* to this distribution.

<img src="histogram.png">


To generate a figure like this, the following function, **density_plot** does this for any feature and label. The first line adds an interactive component that lets you choose these parameters using sliders. 

In [5]:
@interact_manual( feature=IntSlider(0,0,12), label=IntSlider(0,1,3) )
def density_plot(feature, label):
    plt.hist(trainx[trainy==label,feature], normed=True)
    
    mu = np.mean(trainx[trainy==label,feature]) # mean
    var = np.var(trainx[trainy==label,feature]) # variance
    std = np.sqrt(var) # standard deviation
    
    x_axis = np.linspace(mu - 3*std, mu + 3*std, 1000)
    plt.plot(x_axis, norm.pdf(x_axis,mu,std), 'r', lw=3)
    plt.title("Winery "+ str(label) )
    plt.xlabel(featurenames[feature], fontsize=14, color='red')
    plt.ylabel('Density', fontsize=14, color='red')
    plt.show()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0wLCBkZXNjcmlwdGlvbj11J2ZlYXR1cmUnLCBtYXg9MTIpLCBJbnRTbGlkZXIodmFsdWU9MSwgZGVzY3JpcHRpb249dSdsYWLigKY=


## Fit a Gaussian to each class

Let's define a function that will fit a Gaussian generative model to the three classes, restricted to just a single feature.

In [6]:
# Assumes y takes on values 1,2,3
def fit_generative_model(x,y,feature):
    k = 3 # number of classes
    mu = np.zeros(k+1) # list of means
    var = np.zeros(k+1) # list of variances
    pi = np.zeros(k+1) # list of class weights
    for label in range(1,k+1):
        indices = (y==label)
        mu[label] = np.mean(x[indices,feature])
        var[label] = np.var(x[indices,feature])
        pi[label] = float(sum(indices))/float(len(y))
    return mu, var, pi

Call this function on the feature 'alcohol'. What are the class weights?

In [7]:
feature = 0 # 'alcohol'
mu, var, pi = fit_generative_model(trainx, trainy, feature)
print pi[1:]

[0.33076923 0.41538462 0.25384615]


Next, display the Gaussian distribution for each of the three classes

In [8]:
@interact_manual( feature=IntSlider(0,0,12) )
def show_densities(feature):
    mu, var, pi = fit_generative_model(trainx, trainy, feature)
    colors = ['r', 'k', 'g']
    
    for label in range(1,4):
        m = mu[label]
        s = np.sqrt(var[label])
        x_axis = np.linspace(m - 3*s, m+3*s, 1000)
        plt.plot(x_axis, norm.pdf(x_axis,m,s), colors[label-1], label="class " + str(label))
        
    plt.xlabel(featurenames[feature], fontsize=14, color='red')
    plt.ylabel('Density', fontsize=14, color='red')
    plt.legend()
    plt.show()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0wLCBkZXNjcmlwdGlvbj11J2ZlYXR1cmUnLCBtYXg9MTIpLCBCdXR0b24oZGVzY3JpcHRpb249dSdSdW4gSW50ZXJhY3QnLCDigKY=


## Predict labels for the test set

How well can we predict the class (1,2,3) based just on one feature? The code below lets us find this out.

In [28]:
@interact( feature=IntSlider(0,0,12) )
def test_model(feature):
    mu, var, pi = fit_generative_model(trainx, trainy, feature)

    k = 3 # Labels 
    n_test = len(testy) # Number of test points
    score = np.zeros((n_test,k+1))
    
    for i in range(0,n_test):
        for label in range(1,k+1):
            score[i,label] = (pi[label]) * norm.pdf(testx[i,feature], mu[label], np.sqrt(var[label])) # or (np.log... + norm.logpdf...)
            
    predictions = np.argmax(score[:,], axis=1)
    
    errors = np.sum(predictions != testy)
    print "Test error using feature " + featurenames[feature] + ": " + str(errors) + "/" + str(n_test)
    
    return round(float(errors)/n_test, 3)

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0wLCBkZXNjcmlwdGlvbj11J2ZlYXR1cmUnLCBtYXg9MTIpLCBPdXRwdXQoKSksIF9kb21fY2xhc3Nlcz0odSd3aWRnZXQtaW7igKY=


## Feature selection

In this notebook, we are looking at classifiers that use just one out of a possible 13 features. Choosing a subset of features is called `feature selection`. In general, this is something we would need to do based solely on the *training set*--that is, without peeking at the *test set*.

For the wine data, we compute the test error associated with each choice of feature.

In [30]:
# Test error
test_error = np.zeros(13)
for feature in range(0,13):
    test_error[feature] = test_model(feature)
test_error

Test error using feature Alcohol: 17/48
Test error using feature Malic acid: 23/48
Test error using feature Ash: 29/48
Test error using feature Alcalinity of ash: 23/48
Test error using feature Magnesium: 21/48
Test error using feature Total phenols: 16/48
Test error using feature Flavanoids: 8/48
Test error using feature Nonflavanoid phenols: 23/48
Test error using feature Proanthocyanins: 16/48
Test error using feature Color intensity: 10/48
Test error using feature Hue: 14/48
Test error using feature OD280/OD315 of diluted wines: 19/48
Test error using feature Proline: 17/48


array([0.354, 0.479, 0.604, 0.479, 0.438, 0.333, 0.167, 0.479, 0.333,
       0.208, 0.292, 0.396, 0.354])

Choosing feature `Flavanoids` results in the lowest test error.